In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
import numpy as np

In [30]:
# a = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True, infer_shape=False)
# a.write(0, [1., 0.])
# a.write(1, [[0., 1.], [1., 2.]])
# b = a.concat()
# tf.reduce_mean(b, 0)


labels = np.zeros((16, 363))
for i in range(16):
    labels[i, np.random.randint(0, 363)] = 1
np.nonzero(labels)

# def negative_sampling(labels, n_sample=32):
#     num_rows, _ = labels.shape
#     sampled_indices = []
    
#     for i in range(num_rows):
#         positive_indices = np.where(labels[i] == 1)[0]
#         negative_indices = np.where(labels[i] == 0)[0]
#         assert len(positive_indices) < n_sample

#         chosen_indices = positive_indices
#         num_negatives_needed = n_sample - len(chosen_indices)
#         chosen_negative_indices = np.random.choice(negative_indices, size=num_negatives_needed, replace=False)
#         chosen_indices = np.concatenate((chosen_indices, chosen_negative_indices))
        
#         # 打乱选择的索引，使正负样本混合
#         sampled_indices.append(chosen_indices)
    
#     batch_index = np.repeat(np.arange(num_rows)[:, None], n_sample, axis=1)
#     return batch_index, np.array(sampled_indices)

def negative_sampling(labels, n_sample=36):
    num_rows, n_cols = labels.shape
    positive_indices = np.unique(np.nonzero(labels)[1])
    negative_indices = np.array([i for i in range(n_cols) if i not in positive_indices])
    num_negatives_needed = n_sample - len(positive_indices)
    chosen_negative_indices = np.random.choice(negative_indices, size=num_negatives_needed, replace=False)
    chosen_indices = np.concatenate((positive_indices, chosen_negative_indices))   
    batch_index = np.repeat(np.arange(num_rows)[:, None], n_sample, axis=1)
    return batch_index, chosen_indices

c = np.random.rand(16, 64, 128)
bi, ri = negative_sampling(labels, 32)
ri

array([ 19,  37,  50,  59,  81,  95, 140, 141, 159, 180, 208, 214, 254,
       271, 326, 346, 286, 283,  70, 182, 345,  88, 337, 103, 131,  76,
       102,  49, 249, 132,  64, 184])

In [21]:

labels = tf.Variable(tf.zeros((16, 363), dtype=tf.int32))
for i in range(16):
    labels[i, np.random.randint(12)].assign(1)
indices = tf.where(labels != 0)[:, 1]
labels[:, tf.where(labels != 0)[:, 1]]


SyntaxError: unexpected EOF while parsing (128688300.py, line 5)

In [44]:
v = tf.Variable(1)
@tf.function
def f(x):
    ta = tf.TensorArray(tf.int32, size=0, dynamic_size=True)
    for i in tf.range(x):
        tf.print("{}".format(i))
        v.assign_add(i)
        ta = ta.write(i, v)
    return ta.stack()

f(5)

Tensor("while/Placeholder:0", shape=(), dtype=int32)
Tensor("while/Placeholder:0", shape=(), dtype=int32)
Tensor("while/Placeholder:0", shape=(), dtype=int32)
Tensor("while/Placeholder:0", shape=(), dtype=int32)
Tensor("while/Placeholder:0", shape=(), dtype=int32)


<tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 1,  2,  4,  7, 11], dtype=int32)>